<a href="https://colab.research.google.com/github/lblogan14/data_structures_and_algorithms/blob/master/ch6_stacks_q_dq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#6.1 Stacks
A **stack** is a collection of objects that are inserted and removed according to the *last-in, first-out (LIFO)* principle.

##6.1.1 The Stack Abstract Data Type
A stack is an abstract data type (ADT) such that an instance `S` supports the following two methods:
* `S.push(e)`: Add element `e` to the top of stack `S`
* `S.pop()`: Remove and return the top element from the stack `S`; an error occurs if the stack is empty

for convenience, the following accessor methods are also defined:
* `S.top()`: Return a reference to the top element of stack `S`, without removing it; an error occurs if the stack is empty
* `S.is_empty()`: Return `True` if stack `S` does not contain any elements
* `len(S)`: Return the number of elements in stack `S`; in Python, this is implemented with the special method `__len__`

##6.1.2 Simple Array-Based Stack Implementation

In [0]:
class ArrayStack:
  '''LIFO Stack implementation using a Python list as underlying storage'''
  
  def __init__(self):
    '''Create an empty stack'''
    self._data = [] # nonpublic list instance
    
  def __len__(self):
    '''Return the number of elements in the stack'''
    return len(self._data)
  
  def is_empty(self):
    '''Return True if the stack is empty'''
    return len(self._data) == 0
  
  def push(self, e):
    '''Add element e to the top of the stack'''
    self._data.append(e) # new item stored at end of list
    
  def top(self):
    '''Return (but do not remove) the element at the top of the stack
    Raise Empty exception if the stack is empty'''
    if self.is_empty():
      raise Empty('Stack is empty')
    return self._data[-1] # the last item in the list
  
  def pop(self):
    '''Remove and return the element from the top of the stack (i.e. LIFO)
    Raise Empty exception if the stack is empty'''
    if self.is_empty():
      raise Empty('Stack is empty')
    return self._data.pop() # remove last item from list

In [0]:
class Empty(Exception):
  '''Error attempting to access an element from an empty container'''
  pass

##6.1.3 Reversing Data Using a Stack
A stack can be used as a general tool to reverse a data sequence.

To reverse the order of lines in a file:

In [0]:
def reverse_file(filename):
  '''Overwrite given file with its contents line-by-line reversed'''
  S = ArrayStack()
  original = open(filename)
  for line in original:
    S.push(line.rstrip('\n')) # re-insert newlines when writing
  original.close()
  
  # now overwrite with contents in LIFO order
  output = open(filename, 'w') # reopen file and overwrite original
  while not S.is_empty():
    output.write(S.pop() + '\n') # re-insert newline characters
  output.close()

##6.1.4 Matching Parentheses and HTML Tags

###Matching Delimiters

In [0]:
def is_matched(expr):
  '''Return True if all delimiters are properly match; False otherwise'''
  lefty = '({[' # opening delimiters
  righty = ')}]' # respective closing delimiters
  S = ArrayStack()
  for c in expr:
    if c in lefty:
      S.push(c) # push left delimiter on stack
    elif c in righty:
      if S.is_empty():
        return False # nothing to match with
      if righty.index(c) != lefty.index(S.pop()):
        return False # mismatched
  return S.is_empty() 

If the length of the original expression is $n$, the algorithm will make at most
$n$ calls to push and $n$ calls to pop. Those calls run in a total of $O(n)$ time, even considering
the amortized nature of the $O(1)$ time bound for those methods

###Matching Tags in a Markup Language

In [0]:
def is_matched_html(raw):
  '''Return True if all HTML tags are properly match; False otherwise'''
  S = ArrayStack()
  j = raw.find('<') # find first '<' character (if any)
  while j != -1:
    k = raw.find('>', j+1) # find next '>' character
    if k == -1:
      return False # invalid tag
    tag = raw[j+1:k] # strip away < >
    if not tag.startwith('/'): # this is opening tag
      S.push(tag)
    else: # this is closing tag
      if S.is_empty():
        return False # nothing to match with
      if tag[1:] != S.pop():
        return False # mismatched delimiter
    j = raw.find('<', k+1) # find next '<' character (if any)
  return S.is_empty()

#6.2 Queues
A **queue** is a collection of objects that are inserted and remove according to the *first-in, first-out (FIFO)* principle.

## 6.2.1 The Queue Abstract Data Type
The queue abstract data type (ADT) supports the following two fundamental methods for a queue `Q`:
* `Q.enqueue(e)`: Add element `e` to the back of queue `Q`
* `Q.dequeue()`: Remove and return the first element from queue `Q`; an error occurs if the queue is empty

Other supporting methods:
* `Q.first()`: Return a reference to the element at the front of queue `Q`,
without removing it; an error occurs if the queue is empty.
* `Q.is_empty()`: Return `True` if queue `Q` does not contain any elements
* `len(Q)`: Return the number of elements in queue `Q`; in Python, `__len__`

##6.2.2 Array-Based Queue Implementation
####A Python Queue Implementation
* `_data`: a reference to a list instance with a fixed capacity
* `_size`: an integer representing the current number of elements stored in the queue (as poopsed to the length of the `_data` list)
* `_front`: an integer that represents the index within `_data` of the first element of the queue (assuming the queue is not empty)

In [0]:
class ArrayQueue:
  '''FIFO queue implementation using a Python list as underlying storage'''
  DEFAULT_CAPACITY = 10 # moderate capacity for all new queues
  
  def __init__(self):
    '''Create an empty queue'''
    self._data = [None] * ArrayQueue.DEFAULT_CAPACITY
    self._size = 0
    self._front = 0
    
  def __len__(self):
    '''Return the number of elements in the queue'''
    return self._size
  
  def is_empty(self):
    '''Return True if the queue is empty'''
    return self._size == 0
  
  def first(self):
    '''Return (but do not remove) the element at the front of the queue
    Raise Empty exception if the queue is empty
    '''
    if self.is_empty():
      raise Empty('Queue is empty')
    return self._data[self._front]
  
  def dequeue(self):
    '''Remove and return the first element of the queue (i.e. FIFO)
    Raise Empty exception if the queue is empty
    '''
    if self.is_empty():
      raise Empty('Queue is empty')
    answer = self._data[self._front]
    self._data[self._front] = None # help garbage collection
    self._front = (self._front + 1)%len(self._data)
    self._size -= 1
    return answer
  
  def enqueue(self, e):
    '''Add an element to the back of queue'''
    if self._size == len(self._data):
      self.resize(2 * len(self._data)) # double the array size
    avail = (self._front + self._size) % len(self._data)
    self._data[avail] = e
    self._size += 1
    
  def _resize(self, cap): # assume cap >= len(self)
    '''Resize to a new list of capacity >= len(self)'''
    old = self._data # keep track of existing list
    self._data = [None] * cap # allocate list with new capacity
    walk = self._front
    for k in range(self._size): # only consider existing elements
      self._data[k] = old[walk] # intentionally shift indices
      walk = (1 + walk) % len(old) # use old size as modulus
    self._front = 0 # front has been realigned

####Adding and Removing Elements
`enqueue` is to add a new element to the back of the queue. Although an instance variable is not maintained for the back of the queue, the location of the next opening is computed based on the formula: \\
`avail = (self._front + self._size) % len(self._data)` \\
The size of the queue is used as it exists prior to the addition of the new element. For example, consider a queue with capacity 10, current size 3, and
first element at index 5. The three elements of such a queue are stored at indices 5,
6, and 7. The new element should be placed at index (front+size) = 8. In a case
with wrap-around, the use of the modular arithmetic achieves the desired circular
semantics. For example, if our hypothetical queue had 3 elements with the first at
index 8, our computation of (8+3) % 10 evaluates to 1, which is perfect since the
three existing elements occupy indices 8, 9, and 0

#6.3 Double-Ended Queues (Deque)
support insertion and deletion at both the front and the back of the queue.

##6.3.1 The Deque Abstract Data
To provide a symmetrical abstraction, the deque ADT is defined so that deque `D` supports:
* `D.add_first(e)`: Add element `e` to the front of deque `D`
* `D.add_last(e)`: Add element `e` to the back of deque `D`
* `D.deleta_first()`: Remove and return the first element from deque `D`; an error occurs if the deque is empty.
* `D.delete_last()`: Remove and return the last element from deque `D`; an error occurs if the deque is empty.

Also,
* `D.first()`: Return (but do not remove) the first element of deque `D`;
an error occurs if the deque is empty.
* `D.last()`: Return (but do not remove) the last element of deque `D`;
an error occurs if the deque is empty.
* `D.is empty()`: Return True if deque `D` does not contain any elements.
* `len(D)`: Return the number of elements in deque `D`; in Python,`__len__` .

##6.3.2 Implementing a Deque with a Circular Array
basically in the same way as the `ArrayQueue` class above.

Whenever
we need to know the index of the back of the deque, or the first available slot
beyond the back of the deque, we use modular arithmetic for the computation. For example, the implementation of the `last()` method uses the index: \\
`back = (self._front + self._size - 1) % len(self._data)` 

One subtlety is that a call to `add_first` method may need to wrap around the
beginning of the array, so we rely on modular arithmetic to circularly *decrement*
the index, as \\
`self._front = (self._front - 1) % len(self._data)  # cyclic shift` 